# LangChain 入門


In [ ]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env", override=True)

## Chat model


### ChatOpenAI の基本的な使い方


In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("こんにちは！私はジョンと言います"),
    AIMessage("こんにちは、ジョンさん！どのようにお手伝いできますか？"),
    HumanMessage("私の名前がわかりますか？"),
]

ai_message = model.invoke(messages)
print(ai_message.content)

### ストリーミング


In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("こんにちは！"),
]

for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

## Prompt template


### ChatPromptTemplate


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
    ]
)

prompt_value = prompt.invoke({"dish": "カレー"})
for message in prompt_value.messages:
    print(f"{message.type}: {message.content}")

### MessagesPlaceholder


In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

prompt_value = prompt.invoke(
    {
        "chat_history": [
            HumanMessage(content="こんにちは！私はジョンと言います！"),
            AIMessage("こんにちは、ジョンさん！どのようにお手伝いできますか？"),
        ],
        "input": "私の名前が分かりますか？",
    }
)
for message in prompt_value.messages:
    print(f"{message.type}: {message.content}")


## Output parser


### PydanticOutputParser を使った Python オブジェクトへの変換


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

_system_prompt = """\
ユーザーが入力した料理のレシピを考えてください。

出力は以下のJSON形式にしてください。
{{
  "ingredients": ["材料1", "材料2"]
  "steps": ["ステップ1", "ステップ2"]
}}
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", _system_prompt),
        ("human", "{dish}"),
    ]
)

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt_value = prompt.invoke({"dish": "カレー"})
ai_message = model.invoke(prompt_value)
print(ai_message.content)

In [ ]:
from pydantic import BaseModel, Field


class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")

In [ ]:
from langchain_core.output_parsers import PydanticOutputParser

output_parser = PydanticOutputParser(pydantic_object=Recipe)

In [ ]:
recipe = output_parser.invoke(ai_message)
print(type(recipe))
print(recipe)

### StrOutputParser


In [ ]:
from langchain_core.messages import AIMessage
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

ai_message = AIMessage(content="こんにちは。私はAIアシスタントです。")
output = output_parser.invoke(ai_message)
print(type(output))
print(output)

## LangChain Expression Language（LCEL）の概要


### prompt と model の連鎖


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [ ]:
chain = prompt | model

In [ ]:
ai_message = chain.invoke({"dish": "カレー"})
print(ai_message.content)

### StrOutputParser を連鎖に追加


In [ ]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()
output = chain.invoke({"dish": "カレー"})
print(output)

### PydanticOutputParser を使う連鎖


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

_system_prompt = """\
ユーザーが入力した料理のレシピを考えてください。

出力は以下のJSON形式にしてください。
{{
  "ingredients": ["材料1", "材料2"]
  "steps": ["ステップ1", "ステップ2"]
}}
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", _system_prompt),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind(
    response_format={"type": "json_object"}
)

In [ ]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")


output_parser = PydanticOutputParser(pydantic_object=Recipe)

In [ ]:
chain = prompt | model | output_parser

In [ ]:
recipe = chain.invoke({"dish": "カレー"})
print(type(recipe))
print(recipe)

### （補足）with_structured_output


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI


class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model="gpt-4o-mini")

chain = prompt | model.with_structured_output(Recipe)

recipe = chain.invoke({"dish": "カレー"})
print(type(recipe))
print(recipe)

# RAG (検索拡張生成) の基礎


## LangChain での RAG の実装をステップバイステップで動かそう


### Document loader


In [ ]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader

loader = DirectoryLoader(
    path="../tmp/langchain",
    glob="**/*.mdx",
    loader_cls=TextLoader,
)

raw_docs = loader.load()
print(len(raw_docs))

### Document transformer


In [ ]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

docs = text_splitter.split_documents(raw_docs)
print(len(docs))

### Embedding model


In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
query = "AWSのS3からデータを読み込むためのDocument loaderはありますか？"

vector = embeddings.embed_query(query)
print(len(vector))
print(vector)

### Vector store


In [ ]:
# 注意:
# このセルの処理は、大勢が同時に実行するとレートリミットのエラーになる可能性があります
# もしもエラーになった場合は、少し時間をおいてもう一度実行してください

from langchain_chroma import Chroma

db = Chroma.from_documents(docs, embeddings)

### Retriever


In [ ]:
retriever = db.as_retriever()

In [ ]:
query = "AWSのS3からデータを読み込むためのDocument loaderはありますか？"

context_docs = retriever.invoke(query)
print(f"len = {len(context_docs)}")

first_doc = context_docs[0]
print(f"metadata = {first_doc.metadata}")
print(first_doc.page_content)

### LCEL を使った RAG の Chain の実装


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

output = chain.invoke(query)
print(output)